In [ ]:
import sys
import os
import numpy as np
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import setup
from methods import grid_selection_classifier3
from sklearn.model_selection import ParameterGrid



def writeResults(datasetID, bestScore, bestParams):
    path = "old_results/batch/fixed/gridsearch_compose.txt"
    file = open(path,"a") 
    string = "{}: {} using {} \n".format(datasetID, bestScore, bestParams)
    file.write(string)
    file.close() 


def main():
    is_windows = sys.platform.startswith('win')
    sep = '\\'
    
    if is_windows == False:
        sep = '/'

    path = os.getcwd()+sep+'data'+sep
    
    #loading datasets
    #99.9736 {'K': 5, 'batches': 100, 'initialLabeledData': 800, 'p': 0.2, 'sizeOfBatch': 152}
    datasets = [setup.loadCHT, setup.load2CDT, setup.load2CHT, setup.load4CR, setup.load4CRE_V1, 
                setup.load4CRE_V2, setup.load5CVT, setup.loadCSurr, setup.load4CE1CF, setup.loadUG_2C_2D, setup.loadMG_2C_2D, 
                setup.loadFG_2C_2D, setup.loadUG_2C_3D, setup.loadUG_2C_5D, setup.loadGEARS_2C_2D, setup.loadCheckerBoard, 
                setup.loadElecData, setup.loadNOAADataset, setup.loadKeystroke]
    batches=100
    '''
    datasets=[setup.loadCSurr]
    arrClfName = ['KNN']#['NB', 'RF', 'LP', 'KNN'] 
    batches=100
    '''
   
    print("**************** BEGIN of COMPOSE GMM results ****************")

    #testing grid search
    for i in range(len(datasets)):
        if i==len(datasets)-2:
            batches=50
        elif i==len(datasets)-1:
            batches=8
        else:
            batches=100

        finalScore = 0
        best_grid={}
        dataValues, dataLabels, description = datasets[i](path, sep)

        initialLabeledData = int(0.05*len(dataLabels))
        sizeOfBatch = int((len(dataLabels)-initialLabeledData)/batches)

        print("{}: {} batches of {} instances".format(description, batches, sizeOfBatch))

        tuned_params = [{"p" : [0.2, 0.3, 0.4, 0.5],
                         "sizeOfBatch":[sizeOfBatch], "batches":[batches],
                         "initialLabeledData":[initialLabeledData]}]
        tuned_params[0].update({"K":[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]})

        for g in ParameterGrid(tuned_params):
            averageAccuracy=0
            gs = grid_selection_classifier3.run(**g)

            try:
                gs.fit(dataValues, dataLabels)
                averageAccuracy = np.mean(gs.predict())
                print(averageAccuracy, g)
                if finalScore < averageAccuracy:
                    finalScore = averageAccuracy
                    best_grid = g
            except Exception:
                print("An error occured in ", description, g)
                #raise Exception

        print(finalScore)
        print(best_grid)
        print("=======================================================================================================")

        writeResults(description, finalScore, best_grid)
    print("******** END of COMPOSE GMM results ********")
    
if __name__ == "__main__":
    main()


C:\Users\taotao\AppData\Roaming\Python\Python36\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.neighbors.kde module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\taotao\AppData\Roaming\Python\Python36\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.semi_supervised.label_propagation module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.semi_supervised. Anything that cannot be imported from sklearn.semi_supervised is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\taotao\AppData\Roaming\Python\Python36\site-packages\sklearn\utils\deprecation.py:144: FutureWarning:

**************** BEGIN of COMPOSE GMM results ****************
One Class Horizontal Translation. 2 Dimensional data.: 100 batches of 152 instances
